# Assignment 3:  Histograms, Data Analysis and Fitting

The file "edmonton.pickle" contains the historical temperature data from Environment Canada, for weather stations that include "Edmonton" in their name( downloaded from climate.weather.gc.ca).  

The data are organized by "station"; a station object contains "name", "latitude", "longitude", "firstYear", "climateID", "stationID", "dates", "minT", "maxT","doy", and "year" and consist of the readings from a single station.

The important objects are "dates", which is an array of datetime objects, "minT", which is a corresponding array of minimum temperature for each date, "maxT" which is the array of maximum temperatures for those dates.  "doy" and "year" refer to "day of the year" (from 0-365) and the year (from 1880-2019).

The snippet of code below will allow you to read in the data.  You will need to download "edmonton.pickle", and "station.py" into your working directory.


In [2]:
import station  
import pickle
import datetime as dt

with open('edmonton.pickle','rb') as f:
    s=pickle.load(f)
    
for station in s:
    print(station.name)
    print(station.maxT)

EDMONTON
[18.9 12.8 16.7 ... 25.6 26.7 21.1]
EDMONTON CITY CENTRE AWOS
[-12.5  -6.1   4.4 ...  16.   14.   18.3]
EDMONTON CALDER
[ 22.8  26.1  18.9  19.4  20.   17.2  16.7  19.4  21.7  24.4  22.8  17.8
  16.1  21.1  16.1  18.3  16.7  19.4  21.1  17.8  26.1  21.7  26.1  27.2
  20.6  16.1  16.7  13.3  20.6  22.8  25.   28.3  30.6  30.6  30.   31.1
  25.   25.   27.2  29.4  31.1  32.2  20.6  25.   25.   22.2  21.7  16.1
  22.8  21.7  21.1  21.1  25.   29.4  25.   26.1  30.   18.9  20.6  20.
  22.2  21.1  24.4  26.1  20.   22.8  26.1  23.3  24.4  21.7  20.   18.9
  21.7  16.1  23.3  16.7  18.9  12.8  13.3  16.1  22.2  23.3  15.6  15.
  14.4  16.1  17.8  25.   22.8  17.8  17.2  14.4  17.2  16.7  15.   17.2
  16.7  19.4  22.2  22.2  17.8  17.2  16.1  23.3  23.9  23.3  22.2  16.1
  10.6  14.4  16.1  22.8  24.4  27.8  24.4  22.8  13.9  15.6  18.9  16.1
  15.   18.9  24.4  25.6  26.1  16.1  14.4  10.    3.3   7.2  10.6   8.9
   8.3  12.8  14.4  15.   10.   12.8  18.3  14.4  13.9  10.6   5.6   6

Look at the data.  

1.  Make plots that show the max and min temperatures as a function of date for a single station
2.  Histogram the max and min temperatures for a single station at a few dates throughout the year. Make sure the histograms have a reasonable number and range for the bins.  
3.  For each pair of stations, find the periods of time during which they both measured temperatures.  If there is an overlapping period, find the mean and standard deviation of the differences between the max and min temperatures measured at the two stations for those periods.  Fill out a table with rows:  Name of Station 1, Name of Station 2, number of days that both measured temperatures, Average Difference in max, standard deviation of difference of Max, Average difference in min, standard deviation of difference in min
4.  For a few of the pairs which have significant differences, make a 2d color histogram of Ta-Tb versus Ta, where Ta and Tb refer to the measurements at the two stations. 

Combine all the data from the different stations into an average max and min for each each day of the year. To do this, you will to pick one station as the standard, and correct each of the others by its average difference.

Keep the data as an np.array.
Plot averages for the year.

## No-atmosphere model of heating and cooling

The atmosphere plays a critical role in weather and climate, but modelling it is difficult.  Anyone who has watched the weather forecast sees weather systems move into and out of regions and winds/fronts/ high and low pressure regions- all atmospheric phenomena.  However, we are going to see whether we can fit the data with a "no-atmosphere" model.  If there were no atmosphere, the the only source of heat would be solar radiation, and the only cooling would be from black body radiation.  We incorporate this as an ordinary differential equation.  

\begin{equation} \frac {dT}{dt}=\alpha F(t) - \beta T^4 \end{equation}

Here $T$ is the (absolute!)temperature since $T^4$ is the Stefan-Boltzman law, $t$ the time, $F$ the solar flux (in Watts/m$^2$) and $\alpha$ and $\beta$ are the parameters of the model that we will allow to fit.  If we were to calculate from first principles, $\alpha$ would include the reflectivity of the surface and the clouds , as well as the heat capacity per square meter of the layer of the earth/atmosphere that heat up and cool down.  Similarly, $\beta$ includes the Stefan-Boltzman constant, the emissivity of the earth, and the heat capacity.  So this model is quite simple.

However, it is still interesting to see how well such a simple average model can work.  Since everything in the model is independent of time (except for the orbit) our model does not allow any differences year to year or any difference between locations at the same latitude.  

To start the problem, I have modified the earth-sun solution from Problem Set 2 to include the rotation of the earth, which is important to calculate the solar flux at Edmonton.    This involves adding the vector from the center of the earth to Edmonton, $\vec{x_{Ed}}$ and solving the differential equation 
\begin{equation} \frac{d\vec{x_{Ed}}}{dt}=\vec{\omega} \times \vec{x_{Ed}}\end{equation} where $\vec{\omega}$ in the constant rotation vector of the earth.  

We start by finding the "north pole vector", which we know is aligned with the earth-sun vector on June 21 and December 21st.  If we look at the Horizon web page for Dec 22, we see that X~0, Y=1 at the solstice, and that the axial tilt (obliquity) is 23.4392911 degrees.  Thus $$\hat{n}=(0,\sin(23.4392911),\cos(23.4392911))\approx(0,0.398,0.918)$$ and $$\vec{\omega}=\frac{2\pi\hat{n}}{24\times 3600 \times 365.2425/366.2425}$$
(Notice here the factor 365.2425/366.2425- which converts from normal days to "sidereal days"; which take into account the fact that the earths revolution around the sun means noon-noon is a little longer than one rotation).

We pick $\vec{x_{Ed}}=\cos(53.55)(1,0,0)+\sin(53.55)\hat{n}$ at the solstice. (Only the latitude matters- in principle we need to set longitude as well, but in our model the longitude doesn't really matter).


We run the code below to generate the inputs to the flux calcuation- the sun-earth vector and the Edmonton vector.  We pick t to cover the year, with say 48 bins per day.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import datetime as dt
import time

GM=132712440041.93938e9 #m^3/s^2
GMEarth=398600.435436e9 #m^3/s^2
MEarth=5.97219e24 #kg
MSun=GM/GMEarth*MEarth  #  this keeps the mass ratio right, with most precise GM values!
AU=149597870700.0 #m (exactly)
D=24*3600.0 #s

#  Factors from Horizon web site at 2458839.916666667 = A.D. 2019-Dec-22 10:00:00.0000 TDB 
#  This is picked to be at the hour closest to the solstice- notice that the x position of earth is very small
x0=np.array([4.858434307587728E-04 *AU, 9.836920738092132E-01 *AU, -4.745398623388847E-05 *AU])
v0=np.array([-1.749013293502635E-02 *AU/D,-5.128449611745279E-05 *AU/D,  4.120640971206839E-07 *AU/D])
tilt=23.4392911/180*np.pi
n=np.array([0, np.sin(tilt),np.cos(tilt)])  # vector of earth's axis
omega=2*np.pi/(D*365.2425/366.2425)*n  #rotation axis
latitude=53.55/180*np.pi  # latitude of the "Edmonton" weather station
radius=6.37e6  #Earth's radius=6370 km from Horizons
x_ed0=radius*(np.array([np.cos(latitude),0,0])+np.sin(latitude)*n)  #Edmonton location at solstice (at least for some year!)
factor=(MSun+MEarth)/MSun  # to convert xe to earth-sun distance
xsun=np.array([0,0,0]) 

solarConstant=1367.6 #W/m**2 from Horizons.  At 1 AU

cm=MEarth*x0/(MEarth+MSun)
x=x0-cm #  x is distance wrt to the cm

vcm=MEarth*v0/(MEarth+MSun)  #velocity of CM
v=v0-vcm


def dvdt(xvArgument,t):
    xv=xvArgument.reshape(3,3)
    xearth_sun=factor*xv[0] #position wrt sun
    distance=np.sqrt(np.dot(xearth_sun,xearth_sun))
    v_ed=np.cross(omega,xv[2])  # velocity of Edmonton is omega x x_ed
    return np.array([xv[1],-GM/distance**3*xearth_sun,v_ed]).reshape(9)

spy=365.2425*24*3600
t=np.linspace(0,380*24*3600,380*48)  # 48 bins per day, starting December 22.  Go 380 days to cover whole next year
y0=np.array([x,v,x_ed0]).reshape(9)

run=True
if run:
    cpuT0=time.process_time()
    ephemeris = odeint(dvdt, y0, t,rtol=1e-12)
    print("CPU Time=",time.process_time()-cpuT0)
    np.save('problem3',ephemeris)
else:
    ephemeris=np.load('problem3.npy')



CPU Time= 31.738442715999998


Write a function to calculate the Flux at any instant of time, from the output of this calculation.  To do this you will need to interpolate ephemeris (since it is only returned at discrete times).  

Ephemeris contains three vectors- the position of the earth wrt to CM, the velocity of the earth wrt CM, and the position of Edmonton with respect to the center of the earth.  

Once we have the three vectors, the flux is
\begin{equation} F(t)=\begin{cases}\phi_0 \frac{\vec{x_{Ed}} \cdot (\vec {x_s}-\vec{x_e})}{|\vec{x_{Ed}}| |\vec {x_s}-\vec{x_e}|} & \text{if } \vec{x_{Ed}} \cdot (\vec {x_s}-\vec{x_e})>0\\0 & \text{if } \vec{x_{Ed}} \cdot (\vec {x_s}-\vec{x_e})<0
\end{cases} \end{equation}

Plot your function versus date (for one year)



Now write a model that integrates the ordinary differential equation:

\begin{equation} \frac {dT}{dt}=\alpha F(t) - \beta T^4 \end{equation}

using F(t) from the calculation of ephemeris.  

You will need to experiment to find values of $\alpha$ and $\beta$ that are semi-reasonable.  

Find the maximum and minimum temperature for each day from this model, and calculate the difference between this model and the data.

Fit the model to the data.  Plot the residuals.  Comment on the reasonableness of the fit.



(Optional) If you would like to experiment some more, we can add some elements to the model and see how the fit improves.  One can add 
feedback("clouds?") by making $\alpha$ and $\beta$ functions (first or second order polynomials) of T. You could also change $T^4$ to $(T-\Delta)^4$- basically saying the cooling infrared radiation doesn't come from the ground, but from higher in the atmosphere where the temperatures are colder.

